Establish project and weights directories

In [ ]:
import sys
import os

# Go up two levels from notebook (Training/GRU) to project root
project_root = os.path.abspath(os.path.join(os.getcwd(), "../.."))
sys.path.append(project_root)

print("Project root added to sys.path:", project_root)

# Ensure the model save directory exists
model_save_path = os.path.join(project_root, 'Models', 'Weights', 'GRU')
os.makedirs(model_save_path, exist_ok=True)  # Creates directory if it doesn't exist

Load and preprocess data & Train the Model with Optuna

In [ ]:
import torch
from Models.GRU import GRUModel   # Import the GRU model
# Import modularized functions
from Training.Helper.dataPreprocessing import load_data, prepare_dataloader, TRAIN_DATA_PATH_1990S
from Training.Helper.PyTorchModular import optuna_tune_and_train


# Load and preprocess data
config = {"use_fft": False, "use_exog": False}

(X_train_seq, y_train_seq, X_valid_seq, y_valid_seq, X_test_seq, y_test_seq, observation_dates, y_scaler) = load_data(
    TRAIN_DATA_PATH_1990S, sequence_length=12, config=config
)

X_train_seq = X_train_seq.reshape(X_train_seq.shape[0], X_train_seq.shape[1], 1)
X_valid_seq = X_valid_seq.reshape(X_valid_seq.shape[0], X_valid_seq.shape[1], 1)

y_train_seq = y_train_seq.reshape(-1, 1)
y_valid_seq = y_valid_seq.reshape(-1, 1)

batch_size = 32
train_loader = prepare_dataloader(X_train_seq, y_train_seq, batch_size=batch_size)
val_loader = prepare_dataloader(X_valid_seq, y_valid_seq, batch_size=batch_size)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Run Optuna & Train Model
model, metadata = optuna_tune_and_train(
    model_class=GRUModel, 
    train_loader=train_loader, 
    val_loader=val_loader, 
    device=device,
    max_epochs=50,
    model_save_path=model_save_path,
    model_name="GRU_inflation",
    use_best_hyperparams=False,  # Set to False to force Optuna tuning every time
    n_trials=20,
    verbose=True
)


Adjustments for 48-Month Forecast & Evaluation

In [ ]:
# --- Make 48-month future predictions using last known sequence ---
import numpy as np
from sklearn.metrics import mean_squared_error
import math
import torch

# Prepare the starting sequence (last sequence in test set)
last_sequence = X_test_seq[-1].reshape(1, 12, 1)  # Shape: (1, 12, 1)
preds = []

model.eval()
with torch.no_grad():
    current_seq = torch.tensor(last_sequence, dtype=torch.float32).to(device)

    for _ in range(48):  # Predict next 48 steps
        next_step = model(current_seq)          # Shape: (1, 1)
        next_value = next_step.item()
        preds.append(next_value)

        # Shift window left and append prediction
        current_input = current_seq.cpu().numpy().reshape(12, 1)  # Always (12, 1)
        next_input = np.append(current_input[1:], [[next_value]], axis=0)  # Shape: (12, 1)
        current_seq = torch.tensor(next_input.reshape(1, 12, 1), dtype=torch.float32).to(device)

# Inverse transform if scaling was applied
if y_scaler is not None:
    preds = y_scaler.inverse_transform(np.array(preds).reshape(-1, 1)).flatten()

# Save predictions
output_path = os.path.join(project_root, "Predictions", "GRU.npy")
np.save(output_path, np.array(preds))
print(f" Saved 48-month forecast to: {output_path}")


# Calculate RMSE
mse = mean_squared_error(y_test_seq, y_pred)
rmse = math.sqrt(mse)
print(f"GRU Test MSE: {mse}")
print(f"GRU Test RMSE: {rmse}")


Make Predictions on Validation Set

*used in standardisation*

In [ ]:
# from Evaluation.Helper.evaluation_helpers import evaluate_model

# # Evaluate Model
# df_comparison, rmse = evaluate_model(
#     model=model, 
#     val_loader=val_loader, 
#     y_scaler=y_scaler, 
#     observation_dates=observation_dates, 
#     device=device,
#     verbose=True
# )